Importation des bibliothèques utiles

In [35]:
import cv2
import numpy as np
import torch
from super_gradients.training import models
from super_gradients.common.object_names import Models
import numpy as np
import math
from DeepSORT.deep_sort_pytorch.utils.parser import get_config
from DeepSORT.deep_sort_pytorch.deep_sort import DeepSort
from tracker_fonction import tracking
from  Yolo_nas_custom_dataset_function import model_import
from  Yolo_nas_custom_dataset_function import crime_detection_yolocustom_model
from datetime import datetime
import sys


Comme vous pouvez le voir nous utilisons super_gradients. En effet, c'est une API développé par DECI qui ont pu développer leur propre modèle YOLO le YOLO-NAS (Network Architecture Search). Pour le moment il est l'état de l'art des modèles YOLO.
Le développement de nouvelles architectures produits par les techniques NAS ont surpassé les architectures conçues par l'homme dans des tâches telles que le traitement du langage naturel, la détection d'objets et la classification d'images.



 L'architecture YOLO-NAS offre des performances élevées en termes d'exactitude pour la détection d'objets sans nécessiter des ressources informatiques élevées. La technologie AutoNAC explore un vaste espace d'architecture pour trouver des équilibres entre la latence et le débit, et la robustesse du modèle YOLO-NAS se manifeste dans sa capacité à gérer diverses conditions et à maintenir une précision élevée

![Description de l'image](Image_video_notebook\benchmark2.jpg)


Pour nos besoins les modèles de tailles M ont le meilleur rapport vitesse/exactitude.Comme on peut le voir sur le graphique le plus rapide et le plus et le plus précis sont le YOLO V6 et le YOLO nas et pour le peu de différence de latence nous avons plutôt penché pour le YOLO NAS

Fonction d'activation (Heatmap) pour que le modèle ne fonctionne que lorsqu'il y a du mouvement sur l'image et ainsi économiser au maximum les ressources de calculs.

In [36]:
# Fonction pour calculer la heatmap
def calculate_activation(frame, prev_frame, activation_threshold=500):
    # Convertir les images en niveaux de gris
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    # Calculer la différence absolue entre les deux images
    diff = cv2.absdiff(gray_frame, gray_prev_frame)

    # Appliquer un seuil pour détecter les changements significatifs
    _, threshold = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # Calculer la somme des pixels pour déterminer l'activation
    activation = np.sum(threshold)
    return activation


Importation du modèle de détection d'agression et du modèle pre-entrainer coco.
Le modèle de YOLO NAS pre-entrainer coco ainsi que celui que nous avons entrainer nous-même sont des modèles de deep Learning.

Ces modèles de réseaux de neuronnes sont composés de différentes couches.

1) Couches de convolution (CNN) Conv2d :

Les couches de convolution sont les composants principaux des réseaux de neurones convolutifs (CNN). Elles sont utilisées pour extraire des caractéristiques des images en appliquant des filtres convolutifs.

2) Couches regroupement (pooling) MaxPool2d :

Les couches de regroupement(pooling) sont utilisées pour réduire la dimensionnalité spatiale des caractéristiques extraites par les couches de convolution, tout en préservant les caractéristiques les plus importantes. Cela permet de réduire le nombre de paramètres et de calculs nécessaires.


3) Couches entièrement connectées (Fully-Connected) Dense:

Les couches entièrement connectées sont utilisées pour effectuer la classification finale des objets détectés. Elles sont souvent utilisées à la fin du réseau pour mapper les caractéristiques extraites vers les classes d'objets détectées.
Fonctions d'activation :

Les fonctions d'activation sont utilisées après chaque couche pour introduire une non-linéarité dans le modèle. Les fonctions d'activation couramment utilisées incluent ReLU (Rectified Linear Activation), Leaky ReLU, et Sigmoid.


Chargement de la vidéo.
A noté que les images sont redimensionner lors du preprocessing à 640x640 pour la couche d'entrer du modèle.

In [37]:
if __name__=="__main__":
    #Charger le modèle
    best_model = model_import()
    device=torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    model=models.get('yolo_nas_m', pretrained_weights="coco").to(device)
    # Charger la vidéo
    video_path = r'C:\Users\davbe\Crime_detection\fighting.gif'
    cap = cv2.VideoCapture(video_path)
    # Obtenir la largeur et la hauteur des frames
    frame_width = int(cap.get(3))  # Width of the frames in the video
    frame_height = int(cap.get(4))  # Height of the frames in the video

    # Lire la première image pour l'utiliser comme référence
    ret, prev_frame = cap.read()
    if not ret:
        print("Erreur de capture vidéo.")
        sys.exit()
    # Create VideoWriter object
    out = cv2.VideoWriter('output_video_vf3.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))
    while True:
        # Lire le frame suivant

        ret, frame = cap.read()
        # Afficher l'image webcam
        #cv2.imshow('Webcam', frame)

        # Attendre 1 milliseconde et vérifier si l'utilisateur appuie sur la touche 'q' pour quitter
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        if not ret:
            break
        if prev_frame is None:
            print("Erreur: Image précédente vide.")
            continue  # ou effectuez une autre action en conséquence
        # Calculer l'activation
        out.write(frame)
        activation = calculate_activation(frame, prev_frame)
        # Utiliser l'activation pour décider d'activer ou non le modèle de détection
        if activation > 500:
            print("première activation 500 à:", datetime.now())
            while (activation>10):
                print("deuxième activation 10 à:", datetime.now())
                ret, frame = cap.read()
                if not ret:
                    print("Erreur de capture vidéo.")
                    sys.exit()
                prediction = crime_detection_yolocustom_model(best_model,frame,ret)
                out.write(frame)
                prediction.save(r'C:\Users\davbe\Crime_detection\crime_detection\resultatdeladetection3.jpg')  # Save as .mp4

                label = prediction[0].prediction.labels
                if len(label) > 0:
                    for i in label:
                        if i == 1:

                            print("activation agression à:", datetime.now())
                            out.write(frame)
                            print("LABEL: ", label)
                            index = np.where(label == 1)[0]
                            print("INDEX: ", index)
                            bboxes = prediction[0].prediction.bboxes_xyxy[index]
                            print("BBOXES: ", bboxes)
                            print("tracking")
                            tracking(cap,model,bboxes)
                            out.write(frame)

                else:
                    # Mettre à jour l'image de référence
                    prev_frame = frame
                    activation = calculate_activation(frame, prev_frame)








    # Libérer les ressources
    cap.release()
    cv2.destroyAllWindows()
print ("fini")



Extracting Dataset Version Zip to crime_detection-1 in yolov5pytorch:: 100%|██████████| 17384/17384 [00:39<00:00, 437.67it/s]


SystemExit: 


Ce code crée un objet VideoWriter de OpenCV pour écrire le résultat des images dans une vidéo.

4) Couches de détection :

Les couches de détection dans le modèle YOLO NAS sont responsables de prédire les boîtes englobantes (bounding boxes) et les scores de confiance pour chaque classe d'objet. Ces couches sont généralement constituées de couches de convolution avec des filtres de taille adaptée pour générer des prédictions de détection à différentes échelles.

Une fois que les prédictions des coordonnées des boîtes englobantes sont générées, elles subissent souvent un traitement supplémentaire pour garantir leur validité dans l'image. Cela peut impliquer l'application de filtres pour éliminer les boîtes ayant une faible probabilité de contenir un objet, ainsi que l'utilisation de techniques de post-traitement telles que la suppression des boîtes redondantes et la suppression des boîtes avec un faible score de confiance. Ce traitement est souvent effectué par une technique appelée "Non-Maximum Suppression" (NMS).

Concernant les prédictions de classification des objets basées sur ces boîtes englobantes, la fonction softmax est utilisée car elle permet de normaliser les scores de classe pour obtenir des probabilités facilitant ainsi l'interprétation des prédictions de classification.

En résumé, les couches de détection des modèles YOLO jouent un rôle crucial dans la génération de prédictions précises de boîtes englobantes et de scores de confiance, tandis que la fonction softmax est souvent utilisée pour normaliser les scores de classe et obtenir des probabilités pour chaque classe d'objet détecté.

![Description de l'image](Image_video_notebook\pred_0.jpg)
